# load pkgs

In [1]:
import PipelineTS
PipelineTS.__version__

'0.3.12'

In [2]:
from PipelineTS.dataset import LoadMessagesSentDataSets
from PipelineTS.io import save_model, load_model

In [3]:
import pandas as pd
time_col = 'date'
target_col = 'ta'

init_data = LoadMessagesSentDataSets()[[time_col, target_col]]
init_data[time_col] = pd.to_datetime(init_data[time_col])
init_data.shape

(602, 2)

In [4]:
accelerator = 'auto'
lags = 60
n = 40
quantile = 0.9

In [5]:
test_data = init_data.iloc[-n:, :]

train_data = init_data.iloc[:-n, :]

train_data.tail(5)

,date,ta
557,2021-07-11,125957
558,2021-07-12,139012
559,2021-07-13,127014
560,2021-07-14,118241
561,2021-07-15,114358


# Preprocessing

In [6]:
from PipelineTS.metrics import quantile_acc
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from PipelineTS.spinesTS.metrics import wmape

from PipelineTS.preprocessing import Scaler

In [7]:
# use a scaler to scale data
valid_data = test_data.copy()
data = train_data.copy()

scaler = Scaler(scaler_name='min_max')

data[target_col] = scaler.fit_transform(train_data[target_col].values.reshape(-1, 1)).squeeze()
valid_data[target_col] = scaler.transform(test_data[target_col].values.reshape(-1, 1)).squeeze()

In [8]:
def inverse_transform_prediction(df, scaler):
    """define a function to inverse data."""
    for i in df.columns:
        if i != time_col:
            df[i] = scaler.inverse_transform(df[i].values.reshape(-1, 1)).squeeze()

In [9]:
def prediction_metrics(true_df, pred_df):
    """define a function to evaluate the performance of model."""
    yt = true_df[target_col].values
    res = {}
    for i in pred_df.columns:
        if i == target_col:
            res['mae'] = mean_absolute_error(yt.reshape(-1, 1), pred_df[i].values.reshape(-1, 1))
            res['mape'] = mean_absolute_percentage_error(yt.reshape(-1, 1), pred_df[i].values.reshape(-1, 1))
            res['wmape'] = wmape(yt.reshape(-1, 1), pred_df[i].values.reshape(-1, 1))
            
    if target_col+'_lower' in pred_df.columns and target_col+'_upper' in pred_df.columns:
        res['quantile_acc'] = quantile_acc(yt, pred_df[target_col+'_lower'].values, pred_df[target_col+'_upper'].values)

    return res

In [10]:
def test_load_and_save_model(fp, model, scaler):
    save_model(fp, model, scaler=scaler)
    model, scaler = load_model(fp)
    res = model.predict(n)
    inverse_transform_prediction(res, scaler)
    print(prediction_metrics(test_data, res))
    return res

# WideGBRTModel

In [11]:
from PipelineTS.ml_model import WideGBRTModel
from xgboost import XGBRegressor

model = WideGBRTModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                          quantile=quantile, estimator=XGBRegressor, extend_daily_target_features=True)
model.fit(data)

In [12]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 41466.52718128562, 'mape': 0.3624444540658178, 'wmape': 0.32866680786667407, 'quantile_acc': 0.575}


,date,ta,ta_lower,ta_upper
0,2021-07-16,152779.260782,113632.575316,191925.946248
1,2021-07-17,228701.474136,163751.758067,293651.190205
2,2021-07-18,207998.712068,150085.065939,265912.358196
3,2021-07-19,201833.891646,146015.430088,257652.353203
4,2021-07-20,197841.782862,143380.085031,252303.480693
5,2021-07-21,183450.615837,133879.920346,233021.311328
6,2021-07-22,182553.509828,133287.706052,231819.313605
7,2021-07-23,176740.735779,129450.469597,224031.001962
8,2021-07-24,178073.843135,130330.505204,225817.181065
9,2021-07-25,168855.486086,124245.111995,213465.860177


In [13]:
test_load_and_save_model('wide_gbrt.zip', model, scaler)

{'mae': 41466.52718128562, 'mape': 0.3624444540658178, 'wmape': 0.32866680786667407, 'quantile_acc': 0.575}


,date,ta,ta_lower,ta_upper
0,2021-07-16,152779.260782,113632.575316,191925.946248
1,2021-07-17,228701.474136,163751.758067,293651.190205
2,2021-07-18,207998.712068,150085.065939,265912.358196
3,2021-07-19,201833.891646,146015.430088,257652.353203
4,2021-07-20,197841.782862,143380.085031,252303.480693
5,2021-07-21,183450.615837,133879.920346,233021.311328
6,2021-07-22,182553.509828,133287.706052,231819.313605
7,2021-07-23,176740.735779,129450.469597,224031.001962
8,2021-07-24,178073.843135,130330.505204,225817.181065
9,2021-07-25,168855.486086,124245.111995,213465.860177


# ProphetModel

In [14]:
from PipelineTS.statistic_model import ProphetModel
model = ProphetModel(time_col=time_col, target_col=target_col, lags=lags, quantile=quantile)

In [15]:
model.fit(data)

In [16]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 20629.31101318634, 'mape': 0.15710029391827462, 'wmape': 0.16350946799935434, 'quantile_acc': 0.825}


,date,ta,ta_lower,ta_upper
0,2021-07-16,120837.635750,88919.855405,152755.416096
1,2021-07-17,124247.455529,91022.245671,157472.665388
2,2021-07-18,116144.893994,86026.454806,146263.333181
3,2021-07-19,129787.642383,94438.154918,165137.129849
4,2021-07-20,133567.600165,96768.760921,170366.439410
5,2021-07-21,127238.696055,92866.552811,161610.839300
6,2021-07-22,126495.277287,92408.183606,160582.370969
7,2021-07-23,118458.236451,87452.790795,149463.682108
8,2021-07-24,121868.056230,89555.181061,154180.931400
9,2021-07-25,113765.494695,84559.390196,142971.599194


In [17]:
test_load_and_save_model('prophet.zip', model, scaler)

{'mae': 20629.31101318634, 'mape': 0.15710029391827462, 'wmape': 0.16350946799935434, 'quantile_acc': 0.825}


,date,ta,ta_lower,ta_upper
0,2021-07-16,120837.635750,88919.855405,152755.416096
1,2021-07-17,124247.455529,91022.245671,157472.665388
2,2021-07-18,116144.893994,86026.454806,146263.333181
3,2021-07-19,129787.642383,94438.154918,165137.129849
4,2021-07-20,133567.600165,96768.760921,170366.439410
5,2021-07-21,127238.696055,92866.552811,161610.839300
6,2021-07-22,126495.277287,92408.183606,160582.370969
7,2021-07-23,118458.236451,87452.790795,149463.682108
8,2021-07-24,121868.056230,89555.181061,154180.931400
9,2021-07-25,113765.494695,84559.390196,142971.599194


# AutoARIMA

In [18]:
from PipelineTS.statistic_model import AutoARIMAModel

In [19]:
model = AutoARIMAModel(time_col=time_col, target_col=target_col, lags=lags, quantile=quantile)
model.fit(data=data)

In [20]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 26842.097945433256, 'mape': 0.23706463101016176, 'wmape': 0.2127524836985068, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,0,129765.757857,111582.766386,147948.749329
1,1,131241.459259,112767.348710,149715.569808
2,2,135444.346075,116141.110791,154747.581358
3,3,136134.389346,116695.025737,155573.752955
4,4,139506.742460,119402.097699,159611.387220
5,5,139653.891874,119520.218205,159787.565544
6,6,142410.544260,121733.051835,163088.036684
7,7,142187.765018,121554.221381,162821.308655
8,8,144483.920625,123397.402761,165570.438490
9,9,144014.180480,123020.330629,165008.030331


In [21]:
test_load_and_save_model('auto_arima.zip', model, scaler)

{'mae': 26842.097945433256, 'mape': 0.23706463101016176, 'wmape': 0.2127524836985068, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,0,129765.757857,111582.766386,147948.749329
1,1,131241.459259,112767.348710,149715.569808
2,2,135444.346075,116141.110791,154747.581358
3,3,136134.389346,116695.025737,155573.752955
4,4,139506.742460,119402.097699,159611.387220
5,5,139653.891874,119520.218205,159787.565544
6,6,142410.544260,121733.051835,163088.036684
7,7,142187.765018,121554.221381,162821.308655
8,8,144483.920625,123397.402761,165570.438490
9,9,144014.180480,123020.330629,165008.030331


# GBDT

In [22]:
from PipelineTS.ml_model import *

## CatBoost

In [23]:
model = CatBoostModel(time_col=time_col, target_col=target_col, lags=30, random_state=42, quantile=quantile)
model.fit(data=data)

In [24]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 31336.234903759963, 'mape': 0.28290025124066653, 'wmape': 0.24837335066311497, 'quantile_acc': 0.375}


,date,ta,ta_lower,ta_upper
0,2021-07-16,142594.803098,118294.980598,166894.625598
1,2021-07-17,140851.167848,116954.870172,164747.465523
2,2021-07-18,144692.676044,119907.348442,169478.003647
3,2021-07-19,153041.676906,126324.162451,179759.191360
4,2021-07-20,147159.892963,121803.583973,172516.201953
5,2021-07-21,142521.701536,118238.796735,166804.606338
6,2021-07-22,159627.049734,131385.500128,187868.599340
7,2021-07-23,155479.357046,128197.696822,182761.017270
8,2021-07-24,151356.867329,125029.263831,177684.470826
9,2021-07-25,146740.268624,121481.072169,171999.465080


In [25]:
test_load_and_save_model('catboost.zip', model, scaler)

{'mae': 31336.234903759963, 'mape': 0.28290025124066653, 'wmape': 0.24837335066311497, 'quantile_acc': 0.375}


,date,ta,ta_lower,ta_upper
0,2021-07-16,142594.803098,118294.980598,166894.625598
1,2021-07-17,140851.167848,116954.870172,164747.465523
2,2021-07-18,144692.676044,119907.348442,169478.003647
3,2021-07-19,153041.676906,126324.162451,179759.191360
4,2021-07-20,147159.892963,121803.583973,172516.201953
5,2021-07-21,142521.701536,118238.796735,166804.606338
6,2021-07-22,159627.049734,131385.500128,187868.599340
7,2021-07-23,155479.357046,128197.696822,182761.017270
8,2021-07-24,151356.867329,125029.263831,177684.470826
9,2021-07-25,146740.268624,121481.072169,171999.465080


## XGBoost

In [26]:
model = XGBoostModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)
model.fit(data=data)

In [27]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 34624.0408203125, 'mape': 0.30387842439836577, 'wmape': 0.274432747215768, 'quantile_acc': 0.375}


,date,ta,ta_lower,ta_upper
0,2021-07-16,124142.789062,103798.226562,144487.375000
1,2021-07-17,144761.984375,119570.507812,169953.468750
2,2021-07-18,149495.890625,123191.632812,175800.156250
3,2021-07-19,147176.281250,121417.289062,172935.281250
4,2021-07-20,130929.562500,108989.640625,152869.468750
5,2021-07-21,148334.734375,122303.414062,174366.046875
6,2021-07-22,143281.593750,118438.117188,168125.078125
7,2021-07-23,162336.359375,133013.718750,191659.000000
8,2021-07-24,153850.000000,126522.234375,181177.781250
9,2021-07-25,176169.390625,143595.046875,208743.734375


In [28]:
test_load_and_save_model('xgboost.zip', model, scaler)

{'mae': 34624.0408203125, 'mape': 0.30387842439836577, 'wmape': 0.274432747215768, 'quantile_acc': 0.375}


,date,ta,ta_lower,ta_upper
0,2021-07-16,124142.789062,103798.226562,144487.375000
1,2021-07-17,144761.984375,119570.507812,169953.468750
2,2021-07-18,149495.890625,123191.632812,175800.156250
3,2021-07-19,147176.281250,121417.289062,172935.281250
4,2021-07-20,130929.562500,108989.640625,152869.468750
5,2021-07-21,148334.734375,122303.414062,174366.046875
6,2021-07-22,143281.593750,118438.117188,168125.078125
7,2021-07-23,162336.359375,133013.718750,191659.000000
8,2021-07-24,153850.000000,126522.234375,181177.781250
9,2021-07-25,176169.390625,143595.046875,208743.734375


## LightGBM

In [29]:
model = LightGBMModel(time_col=time_col, target_col=target_col, lags=n, random_state=42, quantile=None, verbose=-1)
model.fit(data=data)

In [30]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 27077.046318390196, 'mape': 0.2351260867052929, 'wmape': 0.21461470214317263}


,date,ta
0,2021-07-16,145114.368602
1,2021-07-17,134022.768419
2,2021-07-18,168305.454808
3,2021-07-19,151619.534519
4,2021-07-20,131536.611600
5,2021-07-21,137690.307380
6,2021-07-22,155375.775677
7,2021-07-23,131881.043382
8,2021-07-24,147226.214885
9,2021-07-25,140183.085630


In [31]:
test_load_and_save_model('lightgbm.zip', model, scaler)

{'mae': 27077.046318390196, 'mape': 0.2351260867052929, 'wmape': 0.21461470214317263}


,date,ta
0,2021-07-16,145114.368602
1,2021-07-17,134022.768419
2,2021-07-18,168305.454808
3,2021-07-19,151619.534519
4,2021-07-20,131536.611600
5,2021-07-21,137690.307380
6,2021-07-22,155375.775677
7,2021-07-23,131881.043382
8,2021-07-24,147226.214885
9,2021-07-25,140183.085630


## RandomForest

In [32]:
model = RandomForestModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)
model.fit(data=data)

In [33]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 33039.98048888488, 'mape': 0.29682306190333463, 'wmape': 0.26187736609300283, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,144078.240705,121725.219363,166431.262048
1,2021-07-17,142339.410336,120351.405236,164327.415437
2,2021-07-18,147687.040013,124576.457505,170797.622521
3,2021-07-19,150610.920475,126886.555348,174335.285602
4,2021-07-20,150582.860260,126864.385549,174301.334971
5,2021-07-21,154313.539773,129811.918735,178815.160810
6,2021-07-22,152286.679482,128210.538089,176362.820876
7,2021-07-23,151232.699507,127377.810178,175087.588836
8,2021-07-24,151913.629707,127915.799131,175911.460284
9,2021-07-25,154126.679958,129664.284637,178589.075279


In [34]:
test_load_and_save_model('rf.zip', model, scaler)

{'mae': 33039.98048888488, 'mape': 0.29682306190333463, 'wmape': 0.26187736609300283, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,144078.240705,121725.219363,166431.262048
1,2021-07-17,142339.410336,120351.405236,164327.415437
2,2021-07-18,147687.040013,124576.457505,170797.622521
3,2021-07-19,150610.920475,126886.555348,174335.285602
4,2021-07-20,150582.860260,126864.385549,174301.334971
5,2021-07-21,154313.539773,129811.918735,178815.160810
6,2021-07-22,152286.679482,128210.538089,176362.820876
7,2021-07-23,151232.699507,127377.810178,175087.588836
8,2021-07-24,151913.629707,127915.799131,175911.460284
9,2021-07-25,154126.679958,129664.284637,178589.075279


# NN Model

In [11]:
from PipelineTS.nn_model import *

## DLinearModel

In [14]:
model = DLinearModel(time_col=time_col, target_col=target_col, lags=10, random_state=42, quantile=quantile)

In [15]:
model.stl_fit(data)

In [16]:
res = model.stl_predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

AttributeError: 'NoneType' object has no attribute 'set_predict_parameters'

In [38]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 37295.61171875, 'mape': 0.3305806304685669, 'wmape': 0.2956078187461187, 'quantile_acc': 0.325}


,date,ta,ta_lower,ta_upper
0,2021-07-16,128949.164062,110120.492188,147777.843750
1,2021-07-17,121173.328125,103947.312500,138399.343750
2,2021-07-18,119036.164062,102250.632812,135821.703125
3,2021-07-19,149312.281250,126286.632812,172337.906250
4,2021-07-20,152158.593750,128546.312500,175770.875000
5,2021-07-21,154106.203125,130092.507812,178119.906250
6,2021-07-22,144746.671875,122662.031250,166831.296875
7,2021-07-23,151426.796875,127965.335938,174888.250000
8,2021-07-24,151416.859375,127957.460938,174876.281250
9,2021-07-25,165733.203125,139323.109375,192143.296875


In [39]:
test_load_and_save_model('dlinear.zip', model, scaler)

{'mae': 37295.61171875, 'mape': 0.3305806304685669, 'wmape': 0.2956078187461187, 'quantile_acc': 0.325}


,date,ta,ta_lower,ta_upper
0,2021-07-16,128949.164062,110120.492188,147777.843750
1,2021-07-17,121173.328125,103947.312500,138399.343750
2,2021-07-18,119036.164062,102250.632812,135821.703125
3,2021-07-19,149312.281250,126286.632812,172337.906250
4,2021-07-20,152158.593750,128546.312500,175770.875000
5,2021-07-21,154106.203125,130092.507812,178119.906250
6,2021-07-22,144746.671875,122662.031250,166831.296875
7,2021-07-23,151426.796875,127965.335938,174888.250000
8,2021-07-24,151416.859375,127957.460938,174876.281250
9,2021-07-25,165733.203125,139323.109375,192143.296875


##  NLinearModel

In [40]:
model = NLinearModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [41]:
model.fit(data)
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 37330.14375, 'mape': 0.33019139644534007, 'wmape': 0.29588152221857184, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,146164.062500,117308.320312,175019.812500
1,2021-07-17,144569.000000,116137.187500,173000.828125
2,2021-07-18,151201.578125,121006.937500,181396.203125
3,2021-07-19,158202.781250,126147.343750,190258.218750
4,2021-07-20,173455.906250,137346.468750,209565.359375
5,2021-07-21,166265.281250,132066.984375,200463.593750
6,2021-07-22,165571.312500,131557.453125,199585.171875
7,2021-07-23,167622.000000,133063.109375,202180.890625
8,2021-07-24,164007.281250,130409.117188,197605.437500
9,2021-07-25,165456.531250,131473.171875,199439.890625


In [42]:
test_load_and_save_model('nlinear.zip', model, scaler)

{'mae': 37330.14375, 'mape': 0.33019139644534007, 'wmape': 0.29588152221857184, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,146164.062500,117308.320312,175019.812500
1,2021-07-17,144569.000000,116137.187500,173000.828125
2,2021-07-18,151201.578125,121006.937500,181396.203125
3,2021-07-19,158202.781250,126147.343750,190258.218750
4,2021-07-20,173455.906250,137346.468750,209565.359375
5,2021-07-21,166265.281250,132066.984375,200463.593750
6,2021-07-22,165571.312500,131557.453125,199585.171875
7,2021-07-23,167622.000000,133063.109375,202180.890625
8,2021-07-24,164007.281250,130409.117188,197605.437500
9,2021-07-25,165456.531250,131473.171875,199439.890625


##  NBeatsModel 

In [43]:
model = NBeatsModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [44]:
model.fit(data)
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 19584.6986328125, 'mape': 0.16375120592530434, 'wmape': 0.155229791839967, 'quantile_acc': 0.675}


,date,ta,ta_lower,ta_upper
0,2021-07-16,135736.046875,111827.015625,159645.078125
1,2021-07-17,140940.515625,115763.585938,166117.468750
2,2021-07-18,144399.031250,118379.523438,170418.515625
3,2021-07-19,151064.578125,123421.218750,178707.937500
4,2021-07-20,152192.406250,124274.289062,180110.531250
5,2021-07-21,140528.234375,115451.742188,165604.734375
6,2021-07-22,117658.117188,98153.218750,137163.015625
7,2021-07-23,126494.007812,104836.515625,148151.500000
8,2021-07-24,113770.664062,95212.812500,132328.500000
9,2021-07-25,118338.140625,98667.578125,138008.718750


In [45]:
test_load_and_save_model('nbeats.zip', model, scaler)

{'mae': 19584.6986328125, 'mape': 0.16375120592530434, 'wmape': 0.155229791839967, 'quantile_acc': 0.675}


,date,ta,ta_lower,ta_upper
0,2021-07-16,135736.046875,111827.015625,159645.078125
1,2021-07-17,140940.515625,115763.585938,166117.468750
2,2021-07-18,144399.031250,118379.523438,170418.515625
3,2021-07-19,151064.578125,123421.218750,178707.937500
4,2021-07-20,152192.406250,124274.289062,180110.531250
5,2021-07-21,140528.234375,115451.742188,165604.734375
6,2021-07-22,117658.117188,98153.218750,137163.015625
7,2021-07-23,126494.007812,104836.515625,148151.500000
8,2021-07-24,113770.664062,95212.812500,132328.500000
9,2021-07-25,118338.140625,98667.578125,138008.718750


## TCNModel

In [12]:
model = TCNModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [14]:
model.fit(data)
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 35090.74195034504, 'mape': 0.31010540668268416, 'wmape': 0.27813185541368796, 'quantile_acc': 0.4}


,date,ta,ta_lower,ta_upper
0,2021-07-16,140493.001902,114828.730823,166157.272980
1,2021-07-17,175662.002314,141226.066641,210097.937987
2,2021-07-18,187493.994834,150106.986664,224881.003004
3,2021-07-19,157154.996208,127334.979336,186975.013081
4,2021-07-20,146515.997460,119349.502939,173682.491982
5,2021-07-21,138135.997028,113059.597540,163212.396516
6,2021-07-22,137255.001593,112398.335290,162111.667896
7,2021-07-23,142787.998452,116551.321569,169024.675336
8,2021-07-24,149073.999296,121269.501593,176878.496998
9,2021-07-25,158424.002162,128287.476590,188560.527734


In [48]:
test_load_and_save_model('tcn.zip', model, scaler)

{'mae': 34588.218694930525, 'mape': 0.30690195035885065, 'wmape': 0.27414881836036076, 'quantile_acc': 0.475}


,date,ta,ta_lower,ta_upper
0,2021-07-16,140493.001902,114828.730823,166157.272980
1,2021-07-17,175662.002314,141226.066641,210097.937987
2,2021-07-18,187493.994834,150106.986664,224881.003004
3,2021-07-19,157154.996208,127334.979336,186975.013081
4,2021-07-20,146515.997460,119349.502939,173682.491982
5,2021-07-21,138135.997028,113059.597540,163212.396516
6,2021-07-22,137255.001593,112398.335290,162111.667896
7,2021-07-23,142787.998452,116551.321569,169024.675336
8,2021-07-24,149073.999296,121269.501593,176878.496998
9,2021-07-25,158424.002162,128287.476590,188560.527734


## NHitsModel

In [49]:
model = NHitsModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [50]:
model.fit(data)
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 25226.28671875, 'mape': 0.22483657232602852, 'wmape': 0.19994544259599567, 'quantile_acc': 0.525}


,date,ta,ta_lower,ta_upper
0,2021-07-16,129828.273438,108896.765625,150759.796875
1,2021-07-17,132439.375000,110915.289062,153963.453125
2,2021-07-18,142921.609375,119018.671875,166824.546875
3,2021-07-19,144612.968750,120326.203125,168899.750000
4,2021-07-20,158269.671875,130883.632812,185655.718750
5,2021-07-21,151263.343750,125467.335938,177059.359375
6,2021-07-22,143454.890625,119430.937500,167478.843750
7,2021-07-23,137749.640625,115020.445312,160478.843750
8,2021-07-24,123837.632812,104265.640625,143409.625000
9,2021-07-25,126111.648438,106023.585938,146199.718750


In [51]:
test_load_and_save_model('nhits.zip', model, scaler)

{'mae': 25226.28671875, 'mape': 0.22483657232602852, 'wmape': 0.19994544259599567, 'quantile_acc': 0.525}


,date,ta,ta_lower,ta_upper
0,2021-07-16,129828.273438,108896.765625,150759.796875
1,2021-07-17,132439.375000,110915.289062,153963.453125
2,2021-07-18,142921.609375,119018.671875,166824.546875
3,2021-07-19,144612.968750,120326.203125,168899.750000
4,2021-07-20,158269.671875,130883.632812,185655.718750
5,2021-07-21,151263.343750,125467.335938,177059.359375
6,2021-07-22,143454.890625,119430.937500,167478.843750
7,2021-07-23,137749.640625,115020.445312,160478.843750
8,2021-07-24,123837.632812,104265.640625,143409.625000
9,2021-07-25,126111.648438,106023.585938,146199.718750


## TFTModel

In [52]:
model = TFTModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile, accelerator='cpu')
model.fit(data)

In [53]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 34548.78046875, 'mape': 0.30836444085253706, 'wmape': 0.27383622801851687, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,133110.140625,109424.492188,156795.796875
1,2021-07-17,137971.187500,113080.109375,162862.265625
2,2021-07-18,141676.312500,115866.437500,167486.187500
3,2021-07-19,143906.781250,117543.796875,170269.750000
4,2021-07-20,145410.656250,118674.750000,172146.578125
5,2021-07-21,146605.781250,119573.507812,173638.062500
6,2021-07-22,147680.187500,120381.492188,174978.906250
7,2021-07-23,148715.156250,121159.804688,176270.515625
8,2021-07-24,149742.812500,121932.617188,177553.000000
9,2021-07-25,150773.062500,122707.390625,178838.718750


In [54]:
test_load_and_save_model('tft.zip', model, scaler)

{'mae': 34548.78046875, 'mape': 0.30836444085253706, 'wmape': 0.27383622801851687, 'quantile_acc': 0.35}


,date,ta,ta_lower,ta_upper
0,2021-07-16,133110.140625,109424.492188,156795.796875
1,2021-07-17,137971.187500,113080.109375,162862.265625
2,2021-07-18,141676.312500,115866.437500,167486.187500
3,2021-07-19,143906.781250,117543.796875,170269.750000
4,2021-07-20,145410.656250,118674.750000,172146.578125
5,2021-07-21,146605.781250,119573.507812,173638.062500
6,2021-07-22,147680.187500,120381.492188,174978.906250
7,2021-07-23,148715.156250,121159.804688,176270.515625
8,2021-07-24,149742.812500,121932.617188,177553.000000
9,2021-07-25,150773.062500,122707.390625,178838.718750


## GAU

In [55]:
from PipelineTS.nn_model import *

In [62]:
model = GAUModel(time_col=time_col, target_col='ta', lags=lags, random_state=42, 
                     quantile=quantile, accelerator=accelerator, patience=10, verbose=True)
model.fit(data)

[log] INFO - Information about the device used for computation:
MPS  available: True   | MPS  use: True
CUDA available: False  | CUDA use: False
TPU  available: False  | TPU  use: False
CPU  available: True   | CPU  use: False


Epoch 1/1000  
1/1 -  [*lr: 0.0009045] - loss: 0.8519 - mae: 0.8519 - 0.09s/epoch - 0.091s/step
Epoch 2/1000  
1/1 -  [*lr: 0.0006545] - loss: 0.6699 - mae: 0.6699 - 0.04s/epoch - 0.037s/step
Epoch 3/1000  
1/1 -  [*lr: 0.0003455] - loss: 0.5645 - mae: 0.5645 - 0.04s/epoch - 0.036s/step
Epoch 4/1000  
1/1 -  [*lr: 9.55e-05] - loss: 0.4958 - mae: 0.4958 - 0.04s/epoch - 0.036s/step
Epoch 5/1000  
1/1 - [*lr: 0.000000e+00] - loss: 0.4631 - mae: 0.4631 - 0.04s/epoch - 0.037s/step
Epoch 6/1000  
1/1 -  [*lr: 9.55e-05] - loss: 0.4545 - mae: 0.4545 - 0.03s/epoch - 0.035s/step
Epoch 7/1000  
1/1 -  [*lr: 0.0003455] - loss: 0.4545 - mae: 0.4545 - 0.04s/epoch - 0.038s/step
Epoch 8/1000  
1/1 -  [*lr: 0.0006545] - loss: 0.4451 - mae: 0.4451 - 0.04s/epoch - 0.039s/step
Epoch 9/1000  
1/1 -  [*lr: 0.0009045] - loss: 0.4137 - mae: 0.4137 - 0.04s/epoch - 0.039s/step
Epoch 10/1000  
1/1 - loss: 0.3668 - mae: 0.3668 - 0.04s/epoch - 0.037s/step
Epoch 11/1000  
1/1 -  [*lr: 0.0009045] - loss: 0.3236 - ma

In [63]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 41826.38534306213, 'mape': 0.36437020956790517, 'wmape': 0.3315190706761151, 'quantile_acc': 0.525}


,date,ta,ta_lower,ta_upper
0,2021-07-16,193384.001459,142741.640105,244026.362813
1,2021-07-17,175455.135185,130640.913918,220269.356452
2,2021-07-18,180847.179866,134280.165573,227414.194158
3,2021-07-19,121784.453407,94416.971224,149151.935589
4,2021-07-20,139682.653002,106496.999556,172868.306449
5,2021-07-21,172678.962230,128767.192051,216590.732408
6,2021-07-22,126933.469095,97892.195370,155974.742821
7,2021-07-23,138687.663158,105825.451256,171549.875059
8,2021-07-24,160881.331661,120804.619573,200958.043750
9,2021-07-25,191744.036424,141634.778826,241853.294021


In [64]:
test_load_and_save_model('gau.zip', model, scaler)

{'mae': 41826.38534306213, 'mape': 0.36437020956790517, 'wmape': 0.3315190706761151, 'quantile_acc': 0.525}


,date,ta,ta_lower,ta_upper
0,2021-07-16,193384.001459,142741.640105,244026.362813
1,2021-07-17,175455.135185,130640.913918,220269.356452
2,2021-07-18,180847.179866,134280.165573,227414.194158
3,2021-07-19,121784.453407,94416.971224,149151.935589
4,2021-07-20,139682.653002,106496.999556,172868.306449
5,2021-07-21,172678.962230,128767.192051,216590.732408
6,2021-07-22,126933.469095,97892.195370,155974.742821
7,2021-07-23,138687.663158,105825.451256,171549.875059
8,2021-07-24,160881.331661,120804.619573,200958.043750
9,2021-07-25,191744.036424,141634.778826,241853.294021


## StackingRNN

In [65]:
model = StackingRNNModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, accelerator=accelerator, patience=10, verbose=True)
model.fit(data)

[log] INFO - Information about the device used for computation:
MPS  available: True   | MPS  use: True
CUDA available: False  | CUDA use: False
TPU  available: False  | TPU  use: False
CPU  available: True   | CPU  use: False


Epoch 1/3000  
1/1 -  [*lr: 0.0009045] - loss: 0.2616 - mae: 0.2616 - 0.81s/epoch - 0.807s/step
Epoch 2/3000  
1/1 -  [*lr: 0.0006545] - loss: 0.1913 - mae: 0.1913 - 0.03s/epoch - 0.032s/step
Epoch 3/3000  
1/1 -  [*lr: 0.0003455] - loss: 0.1446 - mae: 0.1446 - 0.03s/epoch - 0.026s/step
Epoch 4/3000  
1/1 -  [*lr: 9.55e-05] - loss: 0.1163 - mae: 0.1163 - 0.03s/epoch - 0.028s/step
Epoch 5/3000  
1/1 - [*lr: 0.000000e+00] - loss: 0.1034 - mae: 0.1034 - 0.03s/epoch - 0.025s/step
Epoch 6/3000  
1/1 -  [*lr: 9.55e-05] - loss: 0.0997 - mae: 0.0997 - 0.03s/epoch - 0.025s/step
Epoch 7/3000  
1/1 -  [*lr: 0.0003455] - loss: 0.1001 - mae: 0.1001 - 0.03s/epoch - 0.026s/step
Epoch 8/3000  
1/1 -  [*lr: 0.0006545] - loss: 0.0959 - mae: 0.0959 - 0.03s/epoch - 0.027s/step
Epoch 9/3000  
1/1 -  [*lr: 0.0009045] - loss: 0.0835 - mae: 0.0835 - 0.03s/epoch - 0.029s/step
Epoch 10/3000  
1/1 - loss: 0.0694 - mae: 0.0694 - 0.03s/epoch - 0.027s/step
Epoch 11/3000  
1/1 -  [*lr: 0.0009045] - loss: 0.0724 - ma

In [66]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 25782.425778980927, 'mape': 0.21630158341777922, 'wmape': 0.2043534425439287, 'quantile_acc': 0.5}


,date,ta,ta_lower,ta_upper
0,2021-07-16,138073.647813,115506.410361,160640.885264
1,2021-07-17,131511.271886,110417.926639,152604.617132
2,2021-07-18,138815.381895,116081.552938,161549.210851
3,2021-07-19,149110.249791,124064.220846,174156.278736
4,2021-07-20,157762.616412,130773.288903,184751.943922
5,2021-07-21,155784.458870,129239.420291,182329.497450
6,2021-07-22,142967.389656,119301.030783,166633.748528
7,2021-07-23,137321.905130,114923.507089,159720.303172
8,2021-07-24,119851.061851,101376.568812,138325.554890
9,2021-07-25,124133.747295,104697.374479,143570.120111


In [67]:
test_load_and_save_model('stacking_rnn.zip', model, scaler)

{'mae': 25782.425778980927, 'mape': 0.21630158341777922, 'wmape': 0.2043534425439287, 'quantile_acc': 0.5}


,date,ta,ta_lower,ta_upper
0,2021-07-16,138073.647813,115506.410361,160640.885264
1,2021-07-17,131511.271886,110417.926639,152604.617132
2,2021-07-18,138815.381895,116081.552938,161549.210851
3,2021-07-19,149110.249791,124064.220846,174156.278736
4,2021-07-20,157762.616412,130773.288903,184751.943922
5,2021-07-21,155784.458870,129239.420291,182329.497450
6,2021-07-22,142967.389656,119301.030783,166633.748528
7,2021-07-23,137321.905130,114923.507089,159720.303172
8,2021-07-24,119851.061851,101376.568812,138325.554890
9,2021-07-25,124133.747295,104697.374479,143570.120111


## Time2Vec

In [68]:
model = Time2VecModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, accelerator=accelerator, patience=10, verbose=True)
model.fit(data)

[log] INFO - Information about the device used for computation:
MPS  available: True   | MPS  use: True
CUDA available: False  | CUDA use: False
TPU  available: False  | TPU  use: False
CPU  available: True   | CPU  use: False


Epoch 1/1000  
1/1 -  [*lr: 0.0090451] - loss: 0.1461 - mae: 0.1461 - 0.87s/epoch - 0.872s/step
Epoch 2/1000  
1/1 -  [*lr: 0.0065451] - loss: 0.1178 - mae: 0.1178 - 0.02s/epoch - 0.018s/step
Epoch 3/1000  
1/1 -  [*lr: 0.0034549] - loss: 0.0724 - mae: 0.0724 - 0.02s/epoch - 0.017s/step
Epoch 4/1000  
1/1 -  [*lr: 0.0009549] - loss: 0.0599 - mae: 0.0599 - 0.02s/epoch - 0.018s/step
Epoch 5/1000  
1/1 - [*lr: 0.000000e+00] - loss: 0.0545 - mae: 0.0545 - 0.02s/epoch - 0.016s/step
Epoch 6/1000  
1/1 -  [*lr: 0.0009549] - loss: 0.0527 - mae: 0.0527 - 0.02s/epoch - 0.016s/step
Epoch 7/1000  
1/1 -  [*lr: 0.0034549] - loss: 0.0527 - mae: 0.0527 - 0.02s/epoch - 0.016s/step
Epoch 8/1000  
1/1 -  [*lr: 0.0065451] - loss: 0.0503 - mae: 0.0503 - 0.02s/epoch - 0.016s/step
Epoch 9/1000  
1/1 -  [*lr: 0.0090451] - loss: 0.0441 - mae: 0.0441 - 0.02s/epoch - 0.016s/step
Epoch 10/1000  
1/1 - loss: 0.0466 - mae: 0.0466 - 0.02s/epoch - 0.016s/step
Epoch 11/1000  
1/1 -  [*lr: 0.0090451] - loss: 0.0530 - 

In [69]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 28517.812520340085, 'mape': 0.23935897032388237, 'wmape': 0.2260343232367561, 'quantile_acc': 0.55}


,date,ta,ta_lower,ta_upper
0,2021-07-16,140466.616832,111902.939529,169030.294135
1,2021-07-17,137870.769125,110027.863581,165713.674668
2,2021-07-18,144874.416595,115086.854863,174661.978327
3,2021-07-19,156787.214243,123691.905260,189882.523227
4,2021-07-20,161618.586417,127181.782472,196055.390363
5,2021-07-21,169228.640515,132678.803488,205778.477541
6,2021-07-22,164700.130762,129407.694904,199992.566619
7,2021-07-23,156553.225717,123522.886771,189583.564664
8,2021-07-24,144195.249899,114596.267864,173794.231934
9,2021-07-25,137800.818010,109977.335327,165624.300692


In [70]:
test_load_and_save_model('time2vec.zip', model, scaler)

{'mae': 28517.812520340085, 'mape': 0.23935897032388237, 'wmape': 0.2260343232367561, 'quantile_acc': 0.55}


,date,ta,ta_lower,ta_upper
0,2021-07-16,140466.616832,111902.939529,169030.294135
1,2021-07-17,137870.769125,110027.863581,165713.674668
2,2021-07-18,144874.416595,115086.854863,174661.978327
3,2021-07-19,156787.214243,123691.905260,189882.523227
4,2021-07-20,161618.586417,127181.782472,196055.390363
5,2021-07-21,169228.640515,132678.803488,205778.477541
6,2021-07-22,164700.130762,129407.694904,199992.566619
7,2021-07-23,156553.225717,123522.886771,189583.564664
8,2021-07-24,144195.249899,114596.267864,173794.231934
9,2021-07-25,137800.818010,109977.335327,165624.300692


## TransformerModel

In [71]:
model = TransformerModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                               quantile=quantile, enable_progress_bar=False, enable_model_summary=False)

In [72]:
model.fit(data)

In [73]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 54528.708984375, 'mape': 0.48204861970104174, 'wmape': 0.43219864158466814, 'quantile_acc': 0.275}


,date,ta,ta_lower,ta_upper
0,2021-07-16,172927.593750,135934.312500,209920.875000
1,2021-07-17,180740.375000,141611.468750,219869.281250
2,2021-07-18,189639.906250,148078.296875,231201.500000
3,2021-07-19,190496.546875,148700.781250,232292.296875
4,2021-07-20,184428.781250,144291.640625,224565.906250
5,2021-07-21,181911.656250,142462.578125,221360.734375
6,2021-07-22,190188.218750,148476.734375,231899.703125
7,2021-07-23,186205.984375,145583.031250,226828.906250
8,2021-07-24,174261.187500,136903.359375,211618.984375
9,2021-07-25,178680.515625,140114.656250,217246.343750


In [74]:
test_load_and_save_model('transformer.zip', model, scaler)

{'mae': 54528.708984375, 'mape': 0.48204861970104174, 'wmape': 0.43219864158466814, 'quantile_acc': 0.275}


,date,ta,ta_lower,ta_upper
0,2021-07-16,172927.593750,135934.312500,209920.875000
1,2021-07-17,180740.375000,141611.468750,219869.281250
2,2021-07-18,189639.906250,148078.296875,231201.500000
3,2021-07-19,190496.546875,148700.781250,232292.296875
4,2021-07-20,184428.781250,144291.640625,224565.906250
5,2021-07-21,181911.656250,142462.578125,221360.734375
6,2021-07-22,190188.218750,148476.734375,231899.703125
7,2021-07-23,186205.984375,145583.031250,226828.906250
8,2021-07-24,174261.187500,136903.359375,211618.984375
9,2021-07-25,178680.515625,140114.656250,217246.343750


## TiDE

In [75]:
model = TiDEModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                               quantile=0.9, enable_progress_bar=False, 
                 enable_model_summary=False)

In [76]:
model.fit(data)
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 25339.9080078125, 'mape': 0.21556717242632167, 'wmape': 0.2008460134641228, 'quantile_acc': 0.6}


,date,ta,ta_lower,ta_upper
0,2021-07-16,148364.859375,118694.031250,178035.671875
1,2021-07-17,123778.195312,100693.164062,146863.234375
2,2021-07-18,169536.000000,134194.281250,204877.734375
3,2021-07-19,166289.250000,131817.203125,200761.296875
4,2021-07-20,153590.140625,122519.664062,184660.593750
5,2021-07-21,151640.031250,121091.914062,182188.125000
6,2021-07-22,165002.125000,130874.843750,199129.406250
7,2021-07-23,156030.734375,124306.523438,187754.937500
8,2021-07-24,136318.328125,109874.296875,162762.359375
9,2021-07-25,133861.625000,108075.648438,159647.609375


In [77]:
test_load_and_save_model('tide.zip', model, scaler)

{'mae': 25339.9080078125, 'mape': 0.21556717242632167, 'wmape': 0.2008460134641228, 'quantile_acc': 0.6}


,date,ta,ta_lower,ta_upper
0,2021-07-16,148364.859375,118694.031250,178035.671875
1,2021-07-17,123778.195312,100693.164062,146863.234375
2,2021-07-18,169536.000000,134194.281250,204877.734375
3,2021-07-19,166289.250000,131817.203125,200761.296875
4,2021-07-20,153590.140625,122519.664062,184660.593750
5,2021-07-21,151640.031250,121091.914062,182188.125000
6,2021-07-22,165002.125000,130874.843750,199129.406250
7,2021-07-23,156030.734375,124306.523438,187754.937500
8,2021-07-24,136318.328125,109874.296875,162762.359375
9,2021-07-25,133861.625000,108075.648438,159647.609375


## PatchRNNModel

In [79]:
model = PatchRNNModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, accelerator=accelerator, patience=100, verbose=True)
model.fit(data)

[log] INFO - Information about the device used for computation:
MPS  available: True   | MPS  use: True
CUDA available: False  | CUDA use: False
TPU  available: False  | TPU  use: False
CPU  available: True   | CPU  use: False


Epoch 1/1000  
1/1 -  [*lr: 0.0009045] - loss: 0.3582 - mae: 0.3582 - 2.47s/epoch - 2.472s/step
Epoch 2/1000  
1/1 -  [*lr: 0.0006545] - loss: 0.2078 - mae: 0.2078 - 0.03s/epoch - 0.029s/step
Epoch 3/1000  
1/1 -  [*lr: 0.0003455] - loss: 0.1906 - mae: 0.1906 - 0.03s/epoch - 0.027s/step
Epoch 4/1000  
1/1 -  [*lr: 9.55e-05] - loss: 0.1572 - mae: 0.1572 - 0.03s/epoch - 0.028s/step
Epoch 5/1000  
1/1 - [*lr: 0.000000e+00] - loss: 0.1362 - mae: 0.1362 - 0.03s/epoch - 0.027s/step
Epoch 6/1000  
1/1 -  [*lr: 9.55e-05] - loss: 0.1292 - mae: 0.1292 - 0.03s/epoch - 0.025s/step
Epoch 7/1000  
1/1 -  [*lr: 0.0003455] - loss: 0.1291 - mae: 0.1291 - 0.03s/epoch - 0.027s/step
Epoch 8/1000  
1/1 -  [*lr: 0.0006545] - loss: 0.1229 - mae: 0.1229 - 0.03s/epoch - 0.028s/step
Epoch 9/1000  
1/1 -  [*lr: 0.0009045] - loss: 0.1007 - mae: 0.1007 - 0.03s/epoch - 0.028s/step
Epoch 10/1000  
1/1 - loss: 0.1180 - mae: 0.1180 - 0.03s/epoch - 0.028s/step
Epoch 11/1000  
1/1 -  [*lr: 0.0009045] - loss: 0.1498 - ma

In [80]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 32953.94769093022, 'mape': 0.29458010493592596, 'wmape': 0.2611954636768208, 'quantile_acc': 0.325}


,date,ta,ta_lower,ta_upper
0,2021-07-16,141981.862983,126825.551848,157138.174118
1,2021-07-17,158098.599937,140602.240708,175594.959166
2,2021-07-18,147302.722518,131373.856290,163231.588745
3,2021-07-19,149353.865117,133127.185980,165580.544255
4,2021-07-20,152817.931613,136088.291968,169547.571258
5,2021-07-21,156218.646670,138995.244744,173442.048595
6,2021-07-22,151905.192280,135308.076603,168502.307957
7,2021-07-23,156794.263898,139487.285998,174101.241798
8,2021-07-24,154523.020728,137545.812946,171500.228509
9,2021-07-25,151390.758416,134868.335276,167913.181555


In [81]:
test_load_and_save_model('seg_rnn.zip', model, scaler)

{'mae': 32953.94769093022, 'mape': 0.29458010493592596, 'wmape': 0.2611954636768208, 'quantile_acc': 0.325}


,date,ta,ta_lower,ta_upper
0,2021-07-16,141981.862983,126825.551848,157138.174118
1,2021-07-17,158098.599937,140602.240708,175594.959166
2,2021-07-18,147302.722518,131373.856290,163231.588745
3,2021-07-19,149353.865117,133127.185980,165580.544255
4,2021-07-20,152817.931613,136088.291968,169547.571258
5,2021-07-21,156218.646670,138995.244744,173442.048595
6,2021-07-22,151905.192280,135308.076603,168502.307957
7,2021-07-23,156794.263898,139487.285998,174101.241798
8,2021-07-24,154523.020728,137545.812946,171500.228509
9,2021-07-25,151390.758416,134868.335276,167913.181555


# MultiOutputRegressorModel

In [12]:
from PipelineTS.ml_model import MultiOutputRegressorModel

In [13]:
from xgboost import XGBRegressor

model = MultiOutputRegressorModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=None, estimator=XGBRegressor)
model.fit(data, fit_kwargs={'verbose': False})

In [84]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 54459.23626415506, 'mape': 0.4797642797551974, 'wmape': 0.4316479955879904}


,date,ta
0,2021-07-16,143959.856897
1,2021-07-17,161026.415456
2,2021-07-18,203839.716013
3,2021-07-19,188231.385947
4,2021-07-20,210180.055101
5,2021-07-21,177272.486977
6,2021-07-22,176724.004782
7,2021-07-23,185638.094933
8,2021-07-24,167454.541148
9,2021-07-25,169408.822588


In [85]:
test_load_and_save_model('mor.zip', model, scaler)

{'mae': 54459.23626415506, 'mape': 0.4797642797551974, 'wmape': 0.4316479955879904}


,date,ta
0,2021-07-16,143959.856897
1,2021-07-17,161026.415456
2,2021-07-18,203839.716013
3,2021-07-19,188231.385947
4,2021-07-20,210180.055101
5,2021-07-21,177272.486977
6,2021-07-22,176724.004782
7,2021-07-23,185638.094933
8,2021-07-24,167454.541148
9,2021-07-25,169408.822588


# MultiStepRegressorModel

In [86]:
from PipelineTS.ml_model.multi_output_model import MultiStepRegressorModel
from xgboost import XGBRegressor

model = MultiStepRegressorModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, estimator=XGBRegressor)
model.fit(data, fit_kwargs={'verbose': False})

In [87]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 150375.22253618538, 'mape': 1.2735182165485521, 'wmape': 1.1918853044321056, 'quantile_acc': 0.05}


,date,ta,ta_lower,ta_upper
0,2021-07-16,143959.856897,110969.974449,176949.739345
1,2021-07-17,244720.756778,180479.102462,308962.411093
2,2021-07-18,204509.255908,152739.509377,256279.002439
3,2021-07-19,185199.598253,139418.891286,230980.305221
4,2021-07-20,212546.011809,158283.603320,266808.420297
5,2021-07-21,202093.188022,151072.803619,253113.572424
6,2021-07-22,359912.994441,259943.577307,459882.411575
7,2021-07-23,351862.671024,254390.123918,449335.218131
8,2021-07-24,224617.174418,166610.801550,282623.547287
9,2021-07-25,337705.274354,244623.753246,430786.795462


In [88]:
test_load_and_save_model('msr.zip', model, scaler)

{'mae': 150375.22253618538, 'mape': 1.2735182165485521, 'wmape': 1.1918853044321056, 'quantile_acc': 0.05}


,date,ta,ta_lower,ta_upper
0,2021-07-16,143959.856897,110969.974449,176949.739345
1,2021-07-17,244720.756778,180479.102462,308962.411093
2,2021-07-18,204509.255908,152739.509377,256279.002439
3,2021-07-19,185199.598253,139418.891286,230980.305221
4,2021-07-20,212546.011809,158283.603320,266808.420297
5,2021-07-21,202093.188022,151072.803619,253113.572424
6,2021-07-22,359912.994441,259943.577307,459882.411575
7,2021-07-23,351862.671024,254390.123918,449335.218131
8,2021-07-24,224617.174418,166610.801550,282623.547287
9,2021-07-25,337705.274354,244623.753246,430786.795462


# RegressorChainModel

In [89]:
from PipelineTS.ml_model.multi_output_model import RegressorChainModel
from xgboost import XGBRegressor

model = RegressorChainModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, estimator=XGBRegressor)
model.fit(data, fit_kwargs={'verbose': False})

In [90]:
res = model.predict(n)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 64444.58156114966, 'mape': 0.5644859948200043, 'wmape': 0.5107925921408183, 'quantile_acc': 0.225}


,date,ta,ta_lower,ta_upper
0,2021-07-16,143959.856897,112656.455363,175263.258430
1,2021-07-17,138319.847103,108676.312338,167963.381869
2,2021-07-18,148652.731317,115968.206952,181337.255682
3,2021-07-19,167709.705908,129416.673761,206002.738055
4,2021-07-20,177486.935551,136316.444116,218657.426986
5,2021-07-21,189013.761218,144450.900698,233576.621737
6,2021-07-22,169730.325005,130842.620399,208618.029610
7,2021-07-23,177353.155748,136222.035991,218484.275505
8,2021-07-24,179529.058139,137757.565723,221300.550555
9,2021-07-25,175932.002710,135219.131287,216644.874133


In [91]:
test_load_and_save_model('rc.zip', model, scaler)

{'mae': 64444.58156114966, 'mape': 0.5644859948200043, 'wmape': 0.5107925921408183, 'quantile_acc': 0.225}


,date,ta,ta_lower,ta_upper
0,2021-07-16,143959.856897,112656.455363,175263.258430
1,2021-07-17,138319.847103,108676.312338,167963.381869
2,2021-07-18,148652.731317,115968.206952,181337.255682
3,2021-07-19,167709.705908,129416.673761,206002.738055
4,2021-07-20,177486.935551,136316.444116,218657.426986
5,2021-07-21,189013.761218,144450.900698,233576.621737
6,2021-07-22,169730.325005,130842.620399,208618.029610
7,2021-07-23,177353.155748,136222.035991,218484.275505
8,2021-07-24,179529.058139,137757.565723,221300.550555
9,2021-07-25,175932.002710,135219.131287,216644.874133
